### Importing the necessary libraries

In [1]:
import json
import pandas as pd
from elasticsearch import Elasticsearch
import minsearch
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings("ignore")

### Data Loading

In [2]:
# let's start by loading our documents ids file
with open("documents_with_ids.json", "rb") as file:
    documents = json.load(file)

In [3]:
documents[0]

{'course': 'data-engineering-zoomcamp',
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'id': '7000acaa'}

### Initialising and Populating ElasticSearch

In [4]:
# remember to run `sudo chown -R 1000:1000 /tmp/elasticsearch_data` after docker-compose up
es_client = Elasticsearch("http://localhost:9200")

if es_client.ping():
    print("Connected to ElasticSearch!")
else:
    print("Connection Failed.")

Connected to ElasticSearch!


In [5]:
# now to create a new index as well as defining the index settings

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

index_name = "course-questions"

# lets delete and create a new index if it exists for ease of re-runs
if es_client.indices.exists(index="course-questions"):
    es_client.indices.delete(index="course-questions")

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
# next we simply have to populate our index using the bulk method
# this method is typically used as a pipeline, hence we need to mention `index` as action 
# you can define multiple 
index = {'index':{
    '_index': index_name}
    }

operations = [item for doc in documents for item in (index, doc)]

es_client.bulk(operations=operations)

ObjectApiResponse({'took': 275, 'errors': False, 'items': [{'index': {'_index': 'course-questions', '_id': '2fVKW5IBjGp2rHM3DkVR', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'course-questions', '_id': '2vVKW5IBjGp2rHM3DkVS', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'course-questions', '_id': '2_VKW5IBjGp2rHM3DkVS', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'course-questions', '_id': '3PVKW5IBjGp2rHM3DkVS', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'course-questions', '_id': '3fVKW5IBjGp2rHM3DkVS', '_version': 1

In [10]:
# now to see if the index was indeed created

indices = es_client.indices.get_alias(index="*")

for index in indices:
    print(index)

course-questions


In [12]:
# last step for this function would be to define the search query function for elasticsearch
def elasticsearch_query(query, course):
    seach_query = {
        "size": 5, # five closest responses to query
        "query": {
            "bool":{
                "must": {
                    "multi-match": {
                        "query": query,
                        "fields": ["questions^3","text","section"], # giving more importance to questions
                        "type": "best_fields"
                    }
                },
                "filter":{
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name,
                                body=seach_query)
    
    result_docs = [hit for hit in response['hits']['hits']]

    return result_docs

### Running the GTD in the ElasticSearch Search Function

In [4]:
ground_truth_df = pd.read_csv("ground-truth-data.csv", delimiter=',', engine='python')

ground_truth_df.head()

,Course,document_ID,Question
0,data-engineering-zoomcamp,7000acaa,When will the course start?
1,data-engineering-zoomcamp,7000acaa,How can I subscribe to the course public Googl...
2,data-engineering-zoomcamp,7000acaa,What should I do before the course starts?
3,data-engineering-zoomcamp,7000acaa,How can I join the course Telegram channel?
4,data-engineering-zoomcamp,7000acaa,Where should I register and join the Slack cha...
